In [6]:
from Networks import MLP2
from Trainers import NavierStokes_2D_Trainer
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [7]:
# 初始化模型
input_shape = (2,)  # x,y
output_shape = (3,)  # u,v,p

model = MLP2(input_shape, output_shape, hidden_layers=[64, 64, 64, 64, 64], activation_function="tanh")

In [8]:
nu = 0.01
optimizer = optim.Adam(model.parameters(), lr=1e-3)

trainer = NavierStokes_2D_Trainer(model, nu, optimizer, lambda_pde=1e-3, device='cuda')

In [9]:
# 定义网格范围
x = torch.linspace(0, 2, 64)  # x 范围从 0 到 2，共 64 个点
y = torch.linspace(-0.1, 0.1, 16)  # y 范围从 -0.1 到 0.1，共 16 个点

# 使用 meshgrid 创建 2D 网格
x_grid, y_grid = torch.meshgrid(x, y, indexing='ij')  # 'ij' 保证顺序为 (x, y)

# 将网格展平为一个二维张量 (N, 2)，每一行代表一个 (x, y) 坐标
collocation_points = torch.stack([x_grid.flatten(), y_grid.flatten()], dim=1)

# 生成边界点: y=0.1 和 y=-0.1 时的 (x, y) 点
boundary_points_upper = torch.stack([x, torch.full_like(x, 0.1)], dim=1)  # y = 0.1
boundary_points_lower = torch.stack([x, torch.full_like(x, -0.1)], dim=1)  # y = -0.1

# 生成入口点：x = 0 时，y = linspace(-0.1, 0.1, 16) 对应的 (x, y) 点
boundary_points_inlet = torch.stack([torch.full_like(y, 0.0), y], dim=1)  # x = 0, y 范围

# 合并为边界点
boundary_points = torch.cat([boundary_points_upper, boundary_points_lower, boundary_points_inlet], dim=0)
# 边界条件，u, v 在 y=±0.1 处都为 0；在 x=0 时，u=5.5, v=0, p=None
boundary_values_u = torch.zeros_like(boundary_points[:, 0])  # u = 0
boundary_values_v = torch.zeros_like(boundary_points[:, 0])  # v = 0
boundary_values_u[-16:] = 5.5
boundary_values_u = boundary_values_u.unsqueeze(1)  # 变为 (batch_size, 1)
boundary_values_v = boundary_values_v.unsqueeze(1)
boundary_values_p = None

In [13]:
collocation_points

tensor([[ 0.0000, -0.1000],
        [ 0.0000, -0.0867],
        [ 0.0000, -0.0733],
        ...,
        [ 2.0000,  0.0733],
        [ 2.0000,  0.0867],
        [ 2.0000,  0.1000]])

In [10]:
# 开始训练
trainer.train(collocation_points, boundary_points, boundary_values_u, boundary_values_v, boundary_values_p, epochs=10000, save_path='model/PINN.pth')

Training Progress:   0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 1/10000, Total Loss: 3.4618e+00 | PDE Loss: 1.3129e-03 | BC Loss: 3.4618e+00
Epoch 101/10000, Total Loss: 7.0124e-01 | PDE Loss: 1.3765e+02 | BC Loss: 5.6359e-01
Epoch 201/10000, Total Loss: 6.4071e-01 | PDE Loss: 1.6501e+02 | BC Loss: 4.7570e-01
Epoch 301/10000, Total Loss: 3.8676e-01 | PDE Loss: 1.0328e+01 | BC Loss: 3.7644e-01
Epoch 401/10000, Total Loss: 3.7351e-01 | PDE Loss: 1.9008e-01 | BC Loss: 3.7332e-01
Epoch 501/10000, Total Loss: 3.7329e-01 | PDE Loss: 5.7060e-02 | BC Loss: 3.7323e-01
Epoch 601/10000, Total Loss: 3.7320e-01 | PDE Loss: 4.0785e-02 | BC Loss: 3.7316e-01
Epoch 701/10000, Total Loss: 3.7309e-01 | PDE Loss: 3.3544e-02 | BC Loss: 3.7306e-01
Epoch 801/10000, Total Loss: 3.7292e-01 | PDE Loss: 2.8095e-02 | BC Loss: 3.7289e-01
Epoch 901/10000, Total Loss: 3.7258e-01 | PDE Loss: 2.4031e-02 | BC Loss: 3.7255e-01
Epoch 1001/10000, Total Loss: 3.7166e-01 | PDE Loss: 6.1475e-02 | BC Loss: 3.7160e-01
Epoch 1101/10000, Total Loss: 3.6639e-01 | PDE Loss: 3.6055e-01 | 

KeyboardInterrupt: 

In [14]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

trainer = NavierStokes_2D_Trainer(model, nu, optimizer, lambda_pde=1e-7, device='cuda')
trainer.train(collocation_points, boundary_points, boundary_values_u, boundary_values_v, boundary_values_p, epochs=10000, save_path='model/PINN.pth')

Training Progress:   0%|          | 0/10000 [00:00<?, ?it/s]

/home/whw/miniconda3/envs/jupyter_env/lib/python3.9/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([144])) that is different to the input size (torch.Size([1024, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/10000, Total Loss: 2.9877e+00 | PDE Loss: 3.8941e-08 | BC Loss: 2.9877e+00
Epoch 101/10000, Total Loss: 2.9877e+00 | PDE Loss: 7.1274e-08 | BC Loss: 2.9877e+00
Epoch 201/10000, Total Loss: 2.9877e+00 | PDE Loss: 6.5600e-08 | BC Loss: 2.9877e+00
Epoch 301/10000, Total Loss: 2.9877e+00 | PDE Loss: 6.1342e-08 | BC Loss: 2.9877e+00
Epoch 401/10000, Total Loss: 2.9877e+00 | PDE Loss: 5.8509e-08 | BC Loss: 2.9877e+00
Epoch 501/10000, Total Loss: 2.9877e+00 | PDE Loss: 5.6760e-08 | BC Loss: 2.9877e+00
Epoch 601/10000, Total Loss: 2.9877e+00 | PDE Loss: 6.2954e-08 | BC Loss: 2.9877e+00
Epoch 701/10000, Total Loss: 2.9877e+00 | PDE Loss: 6.9850e-08 | BC Loss: 2.9877e+00
Epoch 801/10000, Total Loss: 2.9877e+00 | PDE Loss: 7.3027e-08 | BC Loss: 2.9877e+00
Epoch 901/10000, Total Loss: 2.9877e+00 | PDE Loss: 7.5047e-08 | BC Loss: 2.9877e+00
Epoch 1001/10000, Total Loss: 2.9877e+00 | PDE Loss: 7.6409e-08 | BC Loss: 2.9877e+00
Epoch 1101/10000, Total Loss: 2.9877e+00 | PDE Loss: 7.7436e-08 | 